In [44]:
import requests

def get_commits(user, repo, token, per_page=100, page=1):
    url = f"https://api.github.com/repos/{user}/{repo}/commits"
    headers = {'Authorization': f'token {token}'}
    params = {'per_page': per_page, 'page': page}
    response = requests.get(url, headers=headers, params=params)
    commits = response.json()
    return commits

# Example usage

users_repos = [('Julia-Susser', 'chatGPT-impact-measurement')]
all_commits = []

for user, repo in users_repos:
    page = 1
    while True:
        commits = get_commits(user, repo, token, page=page)
        if not commits:
            break
        all_commits.extend(commits)
        page += 1

print(f'Total commits fetched: {len(all_commits)}')


Total commits fetched: 20


In [35]:
import requests
from datetime import datetime 

def search_repositories(query, token, per_page=10):
    url = "https://api.github.com/search/repositories"
    headers = {'Authorization': f'token {token}'}
    params = {'q': query, 'per_page': per_page}
    response = requests.get(url, headers=headers, params=params)
    return response.json()

def get_contributors(contributors_url, token):
    headers = {'Authorization': f'token {token}'}
    response = requests.get(contributors_url, headers=headers)
    return response.json()

def get_user_details(user_url, token):
    headers = {'Authorization': f'token {token}'}
    response = requests.get(user_url, headers=headers)
    return response.json()

def get_lines_of_code(repo_url, token, username):
    headers = {'Authorization': f'token {token}'}
    response = requests.get(repo_url, headers=headers)
    data = response.json()
    print(data)
    print("\n\n")
    if 'files' in data:
        for file in data['files']:
            if file['author']['login'] == username:
                return file['changes']  # or use file['additions'] + file['deletions'] for total changes
    return 0

# Example usage
query = 'university of pennsylvania "final-project"'

# Search for repositories
repositories = search_repositories(query, token)


# Collect contributors and their timestamps
users = []
for repo in repositories.get('items', []):
    contributors_url = repo['contributors_url']
    contributors = get_contributors(contributors_url, token)
    timestamp = datetime.strptime(repo['updated_at'], "%Y-%m-%dT%H:%M:%SZ")  # Convert timestamp to datetime object
    for contributor in contributors:
        users.append({'contributor': contributor, 'timestamp': timestamp})

# Display user information
for user_info in users:
    contributor = user_info['contributor']
    timestamp = user_info['timestamp']
    print(f"User: {contributor['login']}, Profile URL: {contributor['html_url']}, Contributions: {contributor['contributions']}, Timestamp: {timestamp}")


User: HuYuxin, Profile URL: https://github.com/HuYuxin, Contributions: 47, Timestamp: 2023-01-31 08:14:22
User: akhabbaz, Profile URL: https://github.com/akhabbaz, Contributions: 25, Timestamp: 2023-01-31 08:14:22
User: cloman93, Profile URL: https://github.com/cloman93, Contributions: 32, Timestamp: 2023-09-08 12:13:49
User: mark345, Profile URL: https://github.com/mark345, Contributions: 24, Timestamp: 2023-09-08 12:13:49
User: aharelick, Profile URL: https://github.com/aharelick, Contributions: 9, Timestamp: 2023-09-08 12:13:49
User: zhaozheng-shen, Profile URL: https://github.com/zhaozheng-shen, Contributions: 1, Timestamp: 2022-01-15 22:24:01
User: zhaozheng-shen, Profile URL: https://github.com/zhaozheng-shen, Contributions: 2, Timestamp: 2022-01-15 22:24:00
User: lingbinc, Profile URL: https://github.com/lingbinc, Contributions: 49, Timestamp: 2022-04-25 17:13:16
User: socrl, Profile URL: https://github.com/socrl, Contributions: 40, Timestamp: 2022-04-25 17:13:16
User: kathyxche

In [31]:
users

[{'login': 'HuYuxin',
  'id': 6688900,
  'node_id': 'MDQ6VXNlcjY2ODg5MDA=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/6688900?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/HuYuxin',
  'html_url': 'https://github.com/HuYuxin',
  'followers_url': 'https://api.github.com/users/HuYuxin/followers',
  'following_url': 'https://api.github.com/users/HuYuxin/following{/other_user}',
  'gists_url': 'https://api.github.com/users/HuYuxin/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/HuYuxin/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/HuYuxin/subscriptions',
  'organizations_url': 'https://api.github.com/users/HuYuxin/orgs',
  'repos_url': 'https://api.github.com/users/HuYuxin/repos',
  'events_url': 'https://api.github.com/users/HuYuxin/events{/privacy}',
  'received_events_url': 'https://api.github.com/users/HuYuxin/received_events',
  'type': 'User',
  'site_admin': False,
  'contributions': 47},
 {'login': '

In [49]:
import requests
from datetime import datetime

def get_user_repositories(username, token):
    headers = {'Authorization': f'token {token}'}
    repos_url = f"https://api.github.com/users/{username}/repos"
    response = requests.get(repos_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return []

def get_commits(repo_url, token, username, start_date, end_date):
    headers = {'Authorization': f'token {token}'}
    commits_url = f"{repo_url}/commits"
    params = {'author': username, 'since': start_date, 'until': end_date}
    response = requests.get(commits_url, headers=headers, params=params)
    commits_data = response.json()
    lines_of_code = 0
    print(commits_data)
    for commit in commits_data:
        lines_of_code += commit['stats']['total']
    return lines_of_code

# Example usage
username_to_check = 'Julia-Susser'
start_date = '2023-01-01T00:00:00Z'
end_date = '2024-01-01T00:00:00Z'

# Fetch repositories for the specified user
repositories = get_user_repositories(username_to_check, token)

# Collect lines of code contributed by the specified user within the time period
lines_of_code_by_user = 0
for repo in repositories:
    repo_url = repo['url']
    lines_of_code_by_user += get_commits(repo_url, token, username_to_check, start_date, end_date)

print(f"Lines of code contributed by user '{username_to_check}' between {start_date} and {end_date}: {lines_of_code_by_user}")


[]
[]
[]
[]
[]
[]
[{'sha': '1117adf2a2bef90b85f0c159a31910ea68a8d592', 'node_id': 'C_kwDOKVQGbtoAKDExMTdhZGYyYTJiZWY5MGI4NWYwYzE1OWEzMTkxMGVhNjhhOGQ1OTI', 'commit': {'author': {'name': 'Julia Susser', 'email': '57605923+Julia-Susser@users.noreply.github.com', 'date': '2023-12-01T20:12:11Z'}, 'committer': {'name': 'Julia Susser', 'email': '57605923+Julia-Susser@users.noreply.github.com', 'date': '2023-12-01T20:12:11Z'}, 'message': 'edits', 'tree': {'sha': '82581142d857c4098c2c8e0d4d941540373ae95d', 'url': 'https://api.github.com/repos/Julia-Susser/company-match/git/trees/82581142d857c4098c2c8e0d4d941540373ae95d'}, 'url': 'https://api.github.com/repos/Julia-Susser/company-match/git/commits/1117adf2a2bef90b85f0c159a31910ea68a8d592', 'comment_count': 0, 'verification': {'verified': False, 'reason': 'unsigned', 'signature': None, 'payload': None}}, 'url': 'https://api.github.com/repos/Julia-Susser/company-match/commits/1117adf2a2bef90b85f0c159a31910ea68a8d592', 'html_url': 'https://github.c

KeyError: 'stats'

In [56]:
import requests

def get_all_commits_by_author(repo_url, author, token):
    """Retrieve all commits from a repository made by a specific author."""
    commits = []
    url = f"{repo_url}/commits?author={author}&per_page=100"
    headers = {'Authorization': f'token {token}'}
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            commits.extend(data)
            url = response.links.get('next', {}).get('url', None)
        else:
            print(f"Failed to fetch commits: {response.status_code}")
            break
    return commits

def count_diff_lines(diff_content):
    """Count the added and removed lines from diff content."""
    lines_added = sum(1 for line in diff_content.split('\n') if line.startswith('+') and not line.startswith('+++'))
    lines_removed = sum(1 for line in diff_content.split('\n') if line.startswith('-') and not line.startswith('---'))
    return lines_added, lines_removed

def main(repo_url, author, token):
    total_added = total_removed = 0
    commits = get_all_commits_by_author(repo_url, author, token)
    print(commits)
    return commits


repo_url = 'https://api.github.com/repos/Julia-Susser/company-match'
author = 'Julia-Susser'  # GitHub username of the author
commits = main(repo_url, author, token)


[{'sha': '4c6398716e6aeb1884fa44b41521584c308b3a56', 'node_id': 'C_kwDOKVQGbtoAKDRjNjM5ODcxNmU2YWViMTg4NGZhNDRiNDE1MjE1ODRjMzA4YjNhNTY', 'commit': {'author': {'name': 'Julia Susser', 'email': '57605923+Julia-Susser@users.noreply.github.com', 'date': '2024-01-29T00:54:56Z'}, 'committer': {'name': 'Julia Susser', 'email': '57605923+Julia-Susser@users.noreply.github.com', 'date': '2024-01-29T00:54:56Z'}, 'message': 'throw away old interations', 'tree': {'sha': '9ac31f39a258c5350fd53b385818f25f82680bc8', 'url': 'https://api.github.com/repos/Julia-Susser/company-match/git/trees/9ac31f39a258c5350fd53b385818f25f82680bc8'}, 'url': 'https://api.github.com/repos/Julia-Susser/company-match/git/commits/4c6398716e6aeb1884fa44b41521584c308b3a56', 'comment_count': 0, 'verification': {'verified': False, 'reason': 'unsigned', 'signature': None, 'payload': None}}, 'url': 'https://api.github.com/repos/Julia-Susser/company-match/commits/4c6398716e6aeb1884fa44b41521584c308b3a56', 'html_url': 'https://githu

In [76]:
import requests

def fetch_commit_details(commit_url, token):
    """Fetch details of a specific commit including file changes."""
    headers = {'Authorization': f'token {token}'}
    response = requests.get(commit_url, headers=headers)
    if response.status_code == 200:
        commit_data = response.json()
        return commit_data
    else:
        print(f"Failed to fetch commit: {response.status_code}")
        return None

def calculate_bytes_added(commit_data):
    """Calculate the total bytes added in the commit."""
    total_lines_added = 0
    if commit_data and 'files' in commit_data:
        for file in commit_data['files']:
            # We assume each addition is a newline, which isn't strictly accurate
            # For a more precise measurement, you'd need the actual byte size of added content
            lines_added = file['additions']  # This is the count of lines added, not bytes
            total_lines_added += lines_added
    return total_lines_added

# Example usage
for commit in commits:
    commit_url = commit["url"]
    commit_data = fetch_commit_details(commit_url, token)
    bytes_added = calculate_bytes_added(commit_data)
    print(f"Total lines added in commit: {bytes_added}")


Total bytes added in commit: 0
Total bytes added in commit: 361
Total bytes added in commit: 1168
Total bytes added in commit: 357
Total bytes added in commit: 1
Total bytes added in commit: 230
Total bytes added in commit: 99678
Total bytes added in commit: 0
Total bytes added in commit: 482
Total bytes added in commit: 1195
Total bytes added in commit: 421
Total bytes added in commit: 530
Total bytes added in commit: 1041
Total bytes added in commit: 980
Total bytes added in commit: 370
Total bytes added in commit: 13101
Total bytes added in commit: 10381
Total bytes added in commit: 1716
Total bytes added in commit: 346
Total bytes added in commit: 144
Total bytes added in commit: 1368
Total bytes added in commit: 688
Total bytes added in commit: 2206
Total bytes added in commit: 393
Total bytes added in commit: 2092
Total bytes added in commit: 84
Total bytes added in commit: 854
Total bytes added in commit: 28
Total bytes added in commit: 425
Total bytes added in commit: 2839
Tota